In [1]:
from __future__ import print_function
import numpy as np
import keras
from keras.datasets import reuters, imdb
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, GRU, Dense, Dropout, Activation, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import pandas as pd

Using TensorFlow backend.


In [2]:
data = pd.read_csv('tweets.160k.random.csv', encoding='utf-8')
data.head()

,label,id,date,query,user,text
0,4,1985770747,Sun May 31 17:44:25 PDT 2009,NO_QUERY,vozabala,Getting ready for another week of fun and game...
1,0,2322735567,Wed Jun 24 23:10:08 PDT 2009,NO_QUERY,liannecab,"http://twitpic.com/8cp6u - I want it, sooo bad"
2,0,1972997427,Sat May 30 10:16:49 PDT 2009,NO_QUERY,nadhirarchangel,iloveyousincethe1stgradeitsthefirsttimewemet ...
3,0,2230992481,Thu Jun 18 17:53:46 PDT 2009,NO_QUERY,doughamlin,@extendr I can add :skype links but :aim links...
4,4,2053227537,Sat Jun 06 03:46:32 PDT 2009,NO_QUERY,Mariallama,just woke up at to rain. . . on the plus side ...


In [3]:
data['label'].value_counts()

4    80259
0    79741
Name: label, dtype: int64

In [5]:
#Preprocess our dataset (extract relevant column, tokenize and integer encode into sequences)
vocab_size = 20000
tokenizer = Tokenizer(num_words= vocab_size)
tokenizer.fit_on_texts(data['text'])
sequences = tokenizer.texts_to_sequences(data['text'])
tweets = sequence.pad_sequences(sequences, padding='post', maxlen=50)

In [6]:
#Split dataset to training and test sets
labels = data['label']
labels = labels.replace(4,1) # replace label '4' with '1' to facilitate one-hot encoding
x_train, x_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.2)

In [7]:
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

128000 train sequences
32000 test sequences


In [9]:
#Convert our class labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train) # 2 classes
y_test = keras.utils.to_categorical(y_test)

In [28]:
#Construct a simple RNN model
# model = Sequential()
# model.add(Embedding(vocab_size, 128))
# model.add(SimpleRNN(128))
# model.add(Dense(2))
# model.add(Activation('softmax'))

# Construct a simple LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 128))
model.add(LSTM(128))
model.add(Dense(2))
model.add(Activation('softmax'))

## Construct a simple GRU model
model = Sequential()
model.add(Embedding(vocab_size, 128))
model.add(GRU(128))
model.add(Dense(2))
model.add(Activation('softmax'))

In [29]:
#Define how our model should be trained
model.compile(loss='categorical_crossentropy',
 optimizer='adam',
 metrics=['accuracy'])

In [30]:
#Train our model
history = model.fit(x_train, y_train,
 batch_size=128,
 epochs=3,
 verbose=1,
 validation_split=0.2)

Train on 102400 samples, validate on 25600 samples
Epoch 1/3
102400/102400 [==============================] - 74s 727us/step - loss: 0.6934 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5014
Epoch 2/3
102400/102400 [==============================] - 72s 702us/step - loss: 0.6932 - acc: 0.5022 - val_loss: 0.6932 - val_acc: 0.5014
Epoch 3/3
102400/102400 [==============================] - 71s 692us/step - loss: 0.6932 - acc: 0.5001 - val_loss: 0.6933 - val_acc: 0.4986


In [31]:
#test our model
score = model.evaluate(x_test, y_test,
 batch_size=128, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

32000/32000 [==============================] - 4s 136us/step
Test score: 0.6932984449863434
Test accuracy: 0.496875
